<a href="https://colab.research.google.com/github/buldubu/AutoEncoder/blob/master/AutoEncoder_with_vgg19_(MNSIT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from torch.tensor import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
from __future__ import print_function
from torchvision.datasets.vision import VisionDataset
import warnings
from PIL import Image
import os
import os.path
import numpy as np
import torch
import codecs
from torchvision.datasets.utils import download_url, download_and_extract_archive, extract_archive, \
    makedir_exist_ok, verify_str_arg



class MNIST(VisionDataset):
    """`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where ``MNIST/processed/training.pt``
            and  ``MNIST/processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """

    resources = [
        ("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "f68b3c2dcbeaaa9fbdd348bbdeb94873"),
        ("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "d53e105ee54ea40749a09fcbcd1e9432"),
        ("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "9fb629c4189551a2d022fa330f9573f3"),
        ("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "ec29112dd5afa0611ce80d1b7f02629c")
    ]

    training_file = 'training.pt'
    test_file = 'test.pt'
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data

    def __init__(self, root, train=True, transform=None, target_transform=None,
                 download=False):
        super(MNIST, self).__init__(root, transform=transform,
                                    target_transform=target_transform)
        self.train = train  # training set or test set

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')

        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file
        self.data, self.targets = torch.load(os.path.join(self.processed_folder, data_file))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], int(self.targets[index])

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

    @property
    def raw_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'processed')

    @property
    def class_to_idx(self):
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self):
        return (os.path.exists(os.path.join(self.processed_folder,
                                            self.training_file)) and
                os.path.exists(os.path.join(self.processed_folder,
                                            self.test_file)))

    def download(self):
        """Download the MNIST data if it doesn't exist in processed_folder already."""

        if self._check_exists():
            return

        makedir_exist_ok(self.raw_folder)
        makedir_exist_ok(self.processed_folder)

        # download files
        for url, md5 in self.resources:
            filename = url.rpartition('/')[2]
            download_and_extract_archive(url, download_root=self.raw_folder, filename=filename, md5=md5)

        # process and save as torch files
        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')

    def extra_repr(self):
        return "Split: {}".format("Train" if self.train is True else "Test")

In [0]:
class AutoEncoder(nn.Module):

    def __init__(self, init_weights=True):
        super(AutoEncoder, self).__init__()
        
        self.conva1 = nn.Conv2d(3,64,3, padding=1)
        self.conva2 = nn.Conv2d(64,64,3, padding=1)
        self.poola = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        self.convb1 = nn.Conv2d(64,128,3, padding=1)
        self.convb2 = nn.Conv2d(128,128,3, padding=1)
        self.poolb = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        self.convc1 = nn.Conv2d(128,256,3, padding=1)
        self.convc2 = nn.Conv2d(256,256,3, padding=1)
        self.convc3 = nn.Conv2d(256,256,3, padding=1)
        self.convc4 = nn.Conv2d(256,256,3, padding=1)
        self.poolc = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        self.convd1 = nn.Conv2d(256,512,3, padding=1)
        self.convd2 = nn.Conv2d(512,512,3, padding=1)
        self.convd3 = nn.Conv2d(512,512,3, padding=1)
        self.convd4 = nn.Conv2d(512,512,3, padding=1)
        self.poold = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        
        self.conve1 = nn.Conv2d(512,512,3, padding=1)
        self.conve2 = nn.Conv2d(512,512,3, padding=1)
        self.conve3 = nn.Conv2d(512,512,3, padding=1)
        self.conve4 = nn.Conv2d(512,512,3, padding=1)
        self.poole = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        self.linear = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1024),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1024, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 512 * 7 * 7),
            nn.ReLU(True),
            nn.Dropout(),
        )
        
        self.convTa1 = nn.ConvTranspose2d(64,3,3, padding=1)
        self.convTa2 = nn.ConvTranspose2d(64,64,3, padding=1)
        self.unpoola = nn.MaxUnpool2d(kernel_size=2, stride=2)

        self.convTb1 = nn.ConvTranspose2d(128,64,3, padding=1)
        self.convTb2 = nn.ConvTranspose2d(128,128,3, padding=1)
        self.unpoolb = nn.MaxUnpool2d(kernel_size=2, stride=2)

        self.convTc1 = nn.ConvTranspose2d(256,128,3, padding=1)
        self.convTc2 = nn.ConvTranspose2d(256,256,3, padding=1)
        self.convTc3 = nn.ConvTranspose2d(256,256,3, padding=1)
        self.convTc4 = nn.ConvTranspose2d(256,256,3, padding=1)
        self.unpoolc = nn.MaxUnpool2d(kernel_size=2, stride=2)

        self.convTd1 = nn.ConvTranspose2d(512,256,3, padding=1)
        self.convTd2 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.convTd3 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.convTd4 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.unpoold = nn.MaxUnpool2d(kernel_size=2, stride=2)
        
        self.convTe1 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.convTe2 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.convTe3 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.convTe4 = nn.ConvTranspose2d(512,512,3, padding=1)
        self.unpoole = nn.MaxUnpool2d(kernel_size=2, stride=2)



        self._initialize_weights()

    def forward(self, x):

        x = self.conva1(x)
        x = F.relu(x)
        x = self.conva2(x)
        x = F.relu(x)
        x, inda = self.poola(x)
        
        x = self.convb1(x)
        x = F.relu(x)
        x = self.convb2(x)
        x = F.relu(x)
        x, indb = self.poolb(x)

        x = self.convc1(x)
        x = F.relu(x)
        x = self.convc2(x)
        x = F.relu(x)
        x = self.convc3(x)
        x = F.relu(x)
        x = self.convc4(x)
        x = F.relu(x)
        x, indc = self.poolc(x)
        
        x = self.convd1(x)
        x = F.relu(x)
        x = self.convd2(x)
        x = F.relu(x)
        x = self.convd3(x)
        x = F.relu(x)
        x = self.convd4(x)
        x = F.relu(x)
        x, indd = self.poold(x)
        
        x = self.conve1(x)
        x = F.relu(x)
        x = self.conve2(x)
        x = F.relu(x)
        x = self.conve3(x)
        x = F.relu(x)
        x = self.conve4(x)
        x = F.relu(x)
        x, inde = self.poole(x)
        
        x = torch.flatten(x, 1)
        x = self.linear(x)
        x = x.view(-1,512,7,7)
        
        x = self.unpoole(x, inde)
        x = self.convTe4(x)
        x = F.relu(x)
        x = self.convTe3(x)
        x = F.relu(x)
        x = self.convTe2(x)
        x = F.relu(x)
        x = self.convTe1(x)
        x = F.relu(x)
        
        x = self.unpoold(x, indd)
        x = self.convTd4(x)
        x = F.relu(x)
        x = self.convTd3(x)
        x = F.relu(x)
        x = self.convTd2(x)
        x = F.relu(x)
        x = self.convTd1(x)
        x = F.relu(x)
        
        x = self.unpoolc(x, indc)
        x = self.convTc4(x)
        x = F.relu(x)
        x = self.convTc3(x)
        x = F.relu(x)
        x = self.convTc2(x)
        x = F.relu(x)
        x = self.convTc1(x)
        x = F.relu(x)
        
        x = self.unpoolb(x, indb)
        x = self.convTb2(x)
        x = F.relu(x)
        x = self.convTb1(x)
        x = F.relu(x)
        
        x = self.unpoola(x, inda)
        x = self.convTa2(x)
        x = F.relu(x)
        x = self.convTa1(x)
        x = F.relu(x)
        
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)



In [0]:
model = AutoEncoder().to(device)

In [0]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00003)

In [0]:
transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.Grayscale(num_output_channels=3),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)



In [0]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [0]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        print(inputs, labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

            dataiter = iter(testloader)
            test, adsf = dataiter.next()
            imshow(torchvision.utils.make_grid(test))
            test = test.cuda()
            ouuu = model(test)

            # print images
            imshow(torchvision.utils.make_grid(ouuu.detach().cpu()))

print('Finished Training')